## 資料合併

In [5]:
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# 0. Helper Functions
# --------------------------
def move_column(df, col_to_move, ref_col, position="after"):
    """
    將 col_to_move 移至 ref_col 之前或之後
    position: "before" 或 "after"
    """
    cols = df.columns.tolist()
    if col_to_move in cols and ref_col in cols:
        cols.remove(col_to_move)
        ref_idx = cols.index(ref_col)
        insert_idx = ref_idx if position == "before" else ref_idx + 1
        cols.insert(insert_idx, col_to_move)
    return df[cols]

def convert_date(series):
    """統一解析日期格式"""
    dt = pd.to_datetime(series, format="%Y-%m-%d", errors="coerce")
    dt = dt.combine_first(pd.to_datetime(series, format="%Y/%m/%d", errors="coerce"))
    return dt.dt.strftime("%Y-%m-%d")

def convert_time_to_minutes(time_str):
    """
    將格式如 'x 小時 y 分鐘' 或只有小時部分轉換為分鐘數。
    """
    if isinstance(time_str, str):
        match = re.match(r'(\d+)\s*小時(?:\s*(\d+)\s*分鐘)?', time_str)
        if match:
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            return hours * 60 + minutes
    return np.nan

def parse_duration(duration_str):
    """
    與 convert_time_to_minutes 相似，用於解析「第一段飛行時間」與「第二段飛行時間」
    """
    return convert_time_to_minutes(duration_str)

# --------------------------
# 1. 合併 CSV 檔案（Long Flight 資料）
# --------------------------
def merge_files(file_paths):
    """讀取並合併多個 CSV 檔案，返回合併後的 DataFrame"""
    data_frames = []
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"正在讀取檔案：{file_path}")
            df = pd.read_csv(file_path, low_memory=False)
            if not df.empty:
                data_frames.append(df)
            else:
                print(f"檔案 {file_path} 為空，跳過。")
        else:
            print(f"檔案 {file_path} 不存在，跳過。")
    if data_frames:
        merged_data = pd.concat(data_frames, ignore_index=True)
        print("合併完成。")
        return merged_data
    else:
        print("沒有任何資料可供合併。")
        return pd.DataFrame()

# 設定檔案路徑
base_path = '/Users/yuchingchen/Documents/專題/cleaned_data/data/long'
file_paths = [
    f'{base_path}/sydney.csv',
    f'{base_path}/sydney_business.csv',
    f'{base_path}/zurich.csv',
    f'{base_path}/zurich_business.csv',
    f'{base_path}/losangeles.csv',
    f'{base_path}/losangeles_business.csv',
    f'{base_path}/frankfurt.csv',
    # f'{base_path}/frankfurt_business.csv',
    f'{base_path}/france.csv',
    # f'{base_path}/france_business.csv',
    f'{base_path}/newyork.csv',
    f'{base_path}/newyork_business.csv',
    f'{base_path}/london.csv',
    f'{base_path}/london_business.csv',
]

# 讀取並合併資料
data = merge_files(file_paths)
if data.empty:
    exit("合併資料為空，程式結束。")

# 移除不需要的欄位
data = data.drop(columns=['查詢日期', '最低價格出現日期'], errors="ignore")

# --------------------------
# 2. 出發日期格式統一
# --------------------------
data["出發日期"] = pd.to_datetime(
    data["出發日期"], 
    format="%Y-%m-%d", 
    errors="coerce"
).combine_first(
    pd.to_datetime(data["出發日期"], format="%Y/%m/%d", errors="coerce")
)
data["出發日期"] = data["出發日期"].dt.strftime("%Y-%m-%d")

# --------------------------
# 3. 計算對數變數並移除缺失資料
# --------------------------
# 將數值型態欄位轉為 numeric
for col in ["平均價格", "最低價格", "價格變異", "中位數價格", "最低價格剩餘天數"]:
    data[col] = pd.to_numeric(data[col], errors='coerce')

for col in ["平均價格", "最低價格", "價格變異", "中位數價格", "最低價格剩餘天數"]:
    log_col = col + "_log"
    data[log_col] = data[col].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan).round(2)

required_cols = ["平均價格_log", "最低價格_log", "最低價格剩餘天數_log", "價格變異_log", "中位數價格_log"]
data = data.dropna(subset=required_cols)

# 顯示抵達機場代號種類及數量
print(data['抵達機場代號'].value_counts())

# --------------------------
# 4. 假期判斷
# --------------------------
# 定義各國假期範圍
holidays = {
    "台灣": [("2025-01-20", "2025-01-28")],
    "美國": [("2025-01-18", "2025-01-20"), ("2025-02-15", "2025-02-17")],
    "澳洲": [("2025-01-25", "2025-01-27")],
}

# 抵達機場代號與地區對應
airport_to_region = {
    "LAX": "美國",  # 洛杉磯
    "SYD": "澳洲",  # 雪梨
    "FRA": "德國",  # 法蘭克福
    "CDG": "法國",  # 巴黎
    "JFK": "美國",  # 紐約
    "LHR": "英國",  # 倫敦
    "ZRH": "瑞士"   # 蘇黎世
}

# 轉換「出發日期」為 datetime 格式（方便比較）
data["出發日期"] = pd.to_datetime(data["出發日期"])

def is_holiday(row):
    # 台灣假期（不論目的地，出發地為台灣）
    for start, end in holidays["台灣"]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1
    # 依據抵達機場對應地區判斷其他國家假期
    region = airport_to_region.get(row["抵達機場代號"])
    if region and region in holidays:
        for start, end in holidays[region]:
            if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
                return 1
    return 0

data["假期"] = data.apply(is_holiday, axis=1)
cols = data.columns.tolist()
if '假期' in cols and '艙等' in cols:
    cols.insert(cols.index('艙等') + 1, cols.pop(cols.index('假期')))
data = data[cols]

# --------------------------
# 5. 合併額外地區經濟資料
# --------------------------
additional_data = {
    "Region": ["美國", "澳洲", "德國", "英國", "法國", "瑞士"],
    "經濟指標": [0.5468126, -0.389354, -0.7021, -1.262158, -0.977473, 2.7842724]
}
region_df = pd.DataFrame(additional_data)
data["Region"] = data["抵達機場代號"].map(airport_to_region)
data = data.merge(region_df, how="left", on="Region")

# --------------------------
# 6. 統一「星期」欄位格式（如：週一、星期一 → 星期一）並新增是否為平日欄位
# --------------------------
week_mapping = {
    '週一': '星期一', '周一': '星期一',
    '週二': '星期二', '周二': '星期二',
    '週三': '星期三', '周三': '星期三',
    '週四': '星期四', '周四': '星期四',
    '週五': '星期五', '周五': '星期五',
    '週六': '星期六', '周六': '星期六',
    '週日': '星期日', '周日': '星期日'
}

if '星期' in data.columns:
    data['星期'] = data['星期'].replace(week_mapping)

weekend_days = ["星期六", "星期日"]
data["是否為平日"] = data["星期"].apply(lambda x: 0 if x in weekend_days else 1)

# --------------------------
# 7. 新增機場指標欄位
# --------------------------
airport_index = {
    "LAX": 1.7011215,
    "JFK": 0.5891658,
    "LHR": 1.0504261,
    "CDG": 0.5969613,
    "FRA": 0.1276154,
    "SYD": -1.56618,
    "ZRH": -2.49911
}
data['機場指標'] = data['抵達機場代號'].map(airport_index)
cols = data.columns.tolist()
if '經濟指標' in cols and '機場指標' in cols:
    flight_idx = cols.index('經濟指標')
    cols.insert(flight_idx + 1, cols.pop(cols.index('機場指標')))
data = data[cols]

# --------------------------
# 8. 新增「飛行時間_分鐘」與「停留時間_分鐘」與「實際飛行時間_分鐘」欄位
# --------------------------
data['飛行時間_分鐘'] = data['飛行時間'].apply(convert_time_to_minutes)
data['停留時間_分鐘'] = data['停留時間'].apply(convert_time_to_minutes)
data['飛行時間_分鐘'] = data['飛行時間_分鐘'].fillna(0).astype('int64')
data['停留時間_分鐘'] = data['停留時間_分鐘'].fillna(0).astype('int64')

# 將「飛行時間_分鐘」移至「飛行時間」後面
cols = data.columns.tolist()
if '飛行時間' in cols and '飛行時間_分鐘' in cols:
    flight_idx = cols.index('飛行時間')
    cols.insert(flight_idx + 1, cols.pop(cols.index('飛行時間_分鐘')))
    
# 將「停留時間_分鐘」移至「停留時間」後面
if '停留時間' in cols and '停留時間_分鐘' in cols:
    layover_idx = cols.index('停留時間')
    cols.insert(layover_idx + 1, cols.pop(cols.index('停留時間_分鐘')))
data = data[cols]

# --------------------------
# 新增「實際飛行時間_分鐘」欄位
# --------------------------
data["實際飛行時間_分鐘"] = data["飛行時間_分鐘"] - data["停留時間_分鐘"]

# 將「實際飛行時間_分鐘」移至「飛行時間_分鐘」後面
cols = data.columns.tolist()
if '飛行時間_分鐘' in cols and '實際飛行時間_分鐘' in cols:
    flight_idx = cols.index('飛行時間_分鐘')
    cols.insert(flight_idx + 1, cols.pop(cols.index('實際飛行時間_分鐘')))
data = data[cols]

# --------------------------
# 9. 轉換航空公司組合及航空聯盟欄位型態
# --------------------------
if '航空公司組合' in data.columns:
    data["航空公司組合"] = pd.to_numeric(data["航空公司組合"], errors='coerce').round().astype('Int64')
if '航空聯盟' in data.columns:
    data["航空聯盟"] = pd.to_numeric(data["航空聯盟"], errors='coerce').round().astype('Int64')
    
# --------------------------
# 10. 新增競爭航班數欄位
# --------------------------
data['competing_flights'] = data.groupby(
    ['抵達機場代號', '出發日期', '出發時段']
)['航空公司'].transform('size')
data = data.sort_values(
    by=['抵達機場代號', '出發日期', '出發時段', '航空公司']
)

# --------------------------
# 11. 新增「飛行時間兩段分類」欄位
# --------------------------
def classify_flight_segments(row):
    # 若直飛（停靠站數量為 0），直接回傳 0
    if row['停靠站數量'] == 0:
        return 0
    try:
        first_duration = parse_duration(row["第一段飛行時間"])
        second_duration = parse_duration(row["第二段飛行時間"])
        if first_duration > second_duration:
            return 1
        elif first_duration < second_duration:
            return 2
        else:
            return 1  # 相等則預設 1
    except Exception:
        return np.nan

data['飛行時間兩段分類'] = data.apply(classify_flight_segments, axis=1)
data['飛行時間兩段分類'] = data['飛行時間兩段分類'].fillna(0).astype(int)
data = move_column(data, "飛行時間兩段分類", "第二段飛行時間", position="after")
# --------------------------
# 12. # 停靠站數量轉換為數值型態
# --------------------------
# 將 '需轉機 1 次的航班。' 轉為 1， '直達航班。' 轉為 0
data["停靠站數量"] = data["停靠站數量"].apply(
    lambda x: x if isinstance(x, (int, float)) else (1 if x == '需轉機 1 次的航班。' else (0 if x == '直達航班。' else x))
)

# --------------------------
# 13. 輸出最終處理後的資料
# --------------------------
# 按出發日期與抵達機場代號排序
data = data.sort_values(by=["出發日期", "抵達機場代號"])

output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv'
data.to_csv(output_path, index=False)
print(f"最終處理後的資料已儲存至：{output_path}")

正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/sydney.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/sydney_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/zurich.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/zurich_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/losangeles.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/losangeles_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/frankfurt.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/france.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/newyork.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/newyork_business.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/london.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/long/london_business.csv
合併完成。
抵達機場代號
LHR    17324
LAX    17280
FRA    14295
SYD    13598
C